# 7. LangSmith를 활용한 RAG 애플리케이션 평가


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 7.4. Ragas를 활용한 합성 테스트 데이터 생성


### 패키지 설치


In [ ]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.9.2 numpy==1.25.2

### 검색 대상 문서 로드


In [ ]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

### Ragas를 활용한 합성 테스트 데이터 생성 구현


In [ ]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]

#### 【주의】알려진 오류에 관해

아래 코드에서 gpt-4o를 사용할 경우 OpenAI API의 Usage tier에 따라 RateLimitError가 발생할 수 있다고 보고되었습니다.

OpenAI API의 Usage tier에 관해서는 공식 문서의 다음 페이지를 참조하세요.

https://platform.openai.com/docs/guides/rate-limits/usage-tiers

이 오류가 발생한 경우 다음 중 하나의 대응을 실시하세요.

1. 같은 Tier에서도 gpt-4o보다 레이트 리밋이 높은 gpt-4o-mini를 사용한다
   - 이 경우, 생성되는 합성 테스트 데이터의 품질이 낮아질 것으로 예상됩니다
2. 과금 등을 통해 Tier를 올린다
   - Tier 2에서는 RateLimitError가 발생하지 않는 것을 확인했습니다 (2024년 10월 31일 기준)

##### 2025/3/15 추가

LangChain 문서의 증가로 인해, gpt-4o-mini를 사용하더라도 Tier 1에서는 오류가 발생한다는 보고가 있습니다.

이 경우, GitHub에서 문서를 로드하는 부분에서 다음과 같이 작동이 확인된 버전인 `langchain==0.2.13`을 지정하도록 하세요.

```python
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)
```


In [ ]:
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o"),
    critic_llm=ChatOpenAI(model="gpt-4o"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

In [ ]:
testset.to_pandas()

### LangSmith의 Dataset 생성


In [ ]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

### 합성 테스트 데이터 저장


In [ ]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

In [ ]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)

## 7.5. LangSmith와 Ragas를 활용한 오프라인 평가 구현


### 커스텀 Evaluator 구현


In [ ]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLM과 Embeddings을 Metric에 설정
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragas의 평가 메트릭의 score 메서드로 점수 산출
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate
    for metric in metrics
]

### 추론 함수 구현


In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [ ]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

### 오프라인 평가 구현·실행


In [ ]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)

## LangSmith를 활용한 온라인 평가 구현


### 피드백 버튼을 표시하는 함수 구현


In [ ]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # Good 버튼과 Bad 버튼 준비
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # 클릭 시 실행될 함수 정의
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("피드백을 전송했습니다")

    # 버튼 클릭 시 on_button_clicked 함수 실행
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # 버튼 표시
    display(good_button, bad_button)

### 피드백 버튼 표시


In [ ]:
from langchain_core.tracers.context import collect_runs

# LangSmith의 트레이스 ID(Run ID)를 가져오기 위해 collect_runs 함수 사용
with collect_runs() as runs_cb:
    output = chain.invoke("LangChain의 개요를 알려줘")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)